In [18]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs

// Import Semantic Kernel
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"
using Azure.AI.OpenAI;
var (model, azureEndpoint, apiKey) = Settings.LoadFromFile();


var endpoint = new Uri(azureEndpoint);
var credentials = new Azure.AzureKeyCredential(apiKey);
var openAIClient = new OpenAIClient(endpoint, credentials);

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.12

In [19]:
var system = 
""" 
Você é um assistente virtual que ajuda as pessoas a encontrar um restaurante e fazer uma reserva. Data atual: 2024-02-15
""";

In [20]:
var functionDef = new FunctionDefinition()
        {
            Name = "bookRestaurant",
            Description = "Book a restaurant",
            Parameters = BinaryData.FromObjectAsJson(
            new
            {
                Type = "object",
                Properties = new
                {
                     name = new
                    {
                        Type = "string",
                        Description = "name of the restaurant"
                    },
                    persons = new
                    {
                        Type = "integer",
                        Description = "quantity of persons for the reservation"
                    },
                     date = new
                    {
                        Type = "string",
                        Description = "date for the reservation"
                    }
                },
                Required = new[] { "name", "persons", "date" },
            },
            new JsonSerializerOptions() { PropertyNamingPolicy = JsonNamingPolicy.CamelCase }),
        };

In [21]:
var completionOptions = new ChatCompletionsOptions
{
    MaxTokens=400,
    Temperature=1f,
    FrequencyPenalty=0.0f,
    PresencePenalty=0.0f,
    NucleusSamplingFactor = 0.95f, // Top P
    DeploymentName=model
};

In [22]:
completionOptions.Functions.Add(functionDef);
completionOptions.Messages.Add(new ChatRequestSystemMessage(system));


In [23]:
completionOptions.Messages.Add(new ChatRequestUserMessage("Olá"));
ChatCompletions response = await openAIClient.GetChatCompletionsAsync(completionOptions);


In [24]:
completionOptions.Messages.Add(new ChatRequestAssistantMessage(response.Choices[0].Message.Content));
response.Choices[0].Message.Content



Olá! Como posso ajudá-lo hoje? Você está procurando um restaurante ou precisa fazer uma reserva?

In [25]:
completionOptions.Messages.Add(new ChatRequestUserMessage("Quero um restaurante mexicano em São Paulo"));
ChatCompletions response = await openAIClient.GetChatCompletionsAsync(completionOptions);
completionOptions.Messages.Add(new ChatRequestAssistantMessage(response.Choices[0].Message.Content));
response.Choices[0].Message.Content

São Paulo possui uma diversidade incrível de restaurantes, incluindo várias opções de culinária mexicana. Aqui estão alguns restaurantes mexicanos populares em São Paulo:

1. **Si Señor**: Uma rede de restaurantes mexicanos que oferece um ambiente alegre e pratos típicos, como burritos, tacos, nachos e margaritas.

2. **Hecho en México**: Um espaço acolhedor que serve autênticos pratos mexicanos, acompanhados de uma seleção de tequilas e mezcal.

3. **La Mexicana**: Conhecido por seu ambiente colorido e comida tradicional mexicana. É um ótimo local para experimentar moles e chilaquiles.

4. **Dedo de la Chica**: Uma opção mais moderna e descontraída que mistura a comida de rua mexicana com pratos mais elaborados.

5. **Tollocos**: Famoso entre os paulistanos pelos seus tacos e burritos generosos e a um preço acessível.

6. **El Kabong Grill**: Combina a culinária Tex-Mex com um ambiente descontraído, é conhecido por suas fajitas e margaritas.

Caso algum desses restaurantes lhe interes

In [26]:
completionOptions.Messages.Add(new ChatRequestUserMessage("Reserve no Hecho en México para 4 pessoas, amanhã as 20h00"));
ChatCompletions response = await openAIClient.GetChatCompletionsAsync(completionOptions);
response.Choices[0].Message

Azure.AI.OpenAI.ChatResponseMessage Role assistant Content <null> ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) FunctionCall Azure.AI.OpenAI.FunctionCall Name bookRestaurant Arguments {"name":"Hecho en MÃ©xico","persons":4,"date":"2024-02-16T20:00:00Z"} AzureExtensionsContext <null>

In [27]:
completionOptions.Messages.Add(new ChatRequestAssistantMessage(response.Choices[0].Message));
completionOptions.Messages.Add(new ChatRequestFunctionMessage("bookRestaurant",  "OK"));

In [28]:
ChatCompletions response = await openAIClient.GetChatCompletionsAsync(completionOptions);
response.Choices[0].Message

Azure.AI.OpenAI.ChatResponseMessage Role assistant Content Sua reserva no Hecho en México para 4 pessoas amanhã às 20:00 foi realizada com sucesso. Se você precisar de mais alguma coisa ou quiser alterar sua reserva, estou à disposição para ajudar. Aproveite seu jantar! ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) FunctionCall <null> AzureExtensionsContext <null>